
# Aprendizaje por refuerzo

## Introducción

El problema al que nos enfrentamos en esta sesión es el denominado [*CartPole*](https://www.gymlibrary.dev/environments/classic_control/cart_pole/). Este entorno consiste en un péndulo (*pole*) conectado a un carro (*cart*) que se mueve a lo largo de un eje horizontal con la idea de mantener el péndulo vertical.


<figure>
<center>
<img src="https://static.packt-cdn.com/products/9781789345803/graphics/assets/9170409d-15f1-453b-816a-6f601a89fcf2.png" width="500px" align="center" >
<figcaption>CartPole</figcaption></center>
</figure>

El sistema se controla aplicando una fuerza de -1 o +1 al carro para que se mueva a la izquierda o a la derecha.

Se proporciona una recompensa (*reward*) de 1 por cada instante de tiempo que el péndulo continúa en una posición aproximadamente vertical, es decir, el ángulo entre péndulo y la normal en el carro no supera los 15&deg; en valor absoluto. El episodio de entrenamiento termina cuando el péndulo va más allá de esa tolerancia o el carro sale de su máximo recorrido.


El estado que nos proporciona el entorno contiene 4 valores:
  * Posición del carro
  * Velocidad del carro
  * Ángulo del péndulo
  * Velocidad angular del péndulo

----------------------------------------

Empezaremos instalando las librerías necesarias:

In [1]:
!pip install gym pyvirtualdisplay > /dev/null 2>&1

Además, también haremos uso de una serie de librerías para poder mostrar los resultados como vídeos:

In [2]:
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

import gym
from gym import logger as gymlogger
from gym.wrappers.record_video import RecordVideo
gymlogger.set_level(40) #error only

import math
import glob
import io
import base64
from IPython.display import HTML
from IPython import display as ipythondisplay
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else:
    print("No se encuentra el video")

def wrap_env(env):
  env = RecordVideo(env, './video',  episode_trigger = lambda episode_number: True)
  return env

# Entender el framework *Gym* y el entorno *CartPole*



Se crea una instancia de *Gym* para el caso de *CartPole* (llamado `CartPole-v1` en la librería)
- Típicamente, la instancia se llama `env`

In [3]:
env = gym.make('CartPole-v1')

Comprobamos que el espacio de acciones es discreto y comprende sólo dos estados:

In [4]:
num_estados = env.action_space.n

print("Espacio de acciones: {}".format(num_estados));

Espacio de acciones: 2


Comprobamos ahora que un estado comprende cuatro valores como se ha comentado antes:
- Para ello se puede utilizar el método `reset` que hay disponible en la instancia creada antes
- Fíjate que el resultado se guarda en una variable llamada `observacion`

In [5]:
observacion = env.reset()

print('Estado con {} valores:\n\t - Posición: {}\n\t - Velocidad (carro): {}\n\t - Ángulo: {}\n\t - Velocidad (péndulo) {}'.format(observacion.shape[0],observacion[0],observacion[1],observacion[2],observacion[3]))

Estado con 4 valores:
	 - Posición: 0.03467617183923721
	 - Velocidad (carro): 0.007081504911184311
	 - Ángulo: -0.044916052371263504
	 - Velocidad (péndulo) -0.016867369413375854


Para entender mejor el valor de las variables de las observaciones, podemos analizar cuál es el rango (valor máximo y mínimo) que puede tener cada variable

- Se puede utilizar el atributo `observation_space` que permite recibir información sobre el límite alto (`high`) y bajo (`low`) de los valores con los que tratarás

In [6]:
limite_bajo = env.observation_space.low
limite_alto = env.observation_space.high

# Deja esta parte tal y como está:
print('- Posición mínima: {}\n- Velocidad mínima (carro): {}\n- Ángulo mínimo: {}\n- Velocidad mínima (péndulo) {}'.format(limite_bajo[0],limite_bajo[1],limite_bajo[2],limite_bajo[3]))
print()
print('- Posición máxima: {}\n- Velocidad máxima (carro): {}\n- Ángulo máxima: {}\n- Velocidad máxima (péndulo) {}'.format(limite_alto[0],limite_alto[1],limite_alto[2],limite_alto[3]))

- Posición mínima: -4.800000190734863
- Velocidad mínima (carro): -3.4028234663852886e+38
- Ángulo mínimo: -0.41887903213500977
- Velocidad mínima (péndulo) -3.4028234663852886e+38

- Posición máxima: 4.800000190734863
- Velocidad máxima (carro): 3.4028234663852886e+38
- Ángulo máxima: 0.41887903213500977
- Velocidad máxima (péndulo) 3.4028234663852886e+38


# Política programada a mano

En este apartado vamos a implementar una política básica para ver con detalle cómo funciona el entorno de `Gym` y cómo se puede interactuar con él.

En el código que se ve abajo, se sigue una política completamente aleatoria: la mitad de las veces se mueve hacia la izquierda (0) y la otra mitad hacia la derecha (1).


In [ ]:
import numpy as np

# Comienzo: politica aleatoria
def politica_a_mano(state):
  if np.random.rand() > 0.5: # Al 50 %
    return 0 # Accion (izquierda)
  else:
    return 1 # Accion (derecha)

# ============================

# Montamos el entorno aprovechando el código del principio para poder generar un vídeo con la ejecución:
env = wrap_env(gym.make('CartPole-v1'))

state = env.reset() # Inicialización
acc_reward = 0      # Recompensa acumulada del episodio

# Bucle para cada instante:
it = 0
while True:
  env.render(mode='rgb_array')

  # Acción
  action = politica_a_mano(state)

  # Ejecución:
  next_state, reward, done, _ = env.step(action)

  # Acumulamos la recompensa:
  acc_reward += reward

  # Comprobamos si ha terminado el episodio:
  if done:
    print("Episodio finalizado con una recompensa de: {}".format(acc_reward))
    break
  else:
    state = next_state

env.close()

# Visualiza el último episodio (desde el env.reset() hasta el env.close())
show_video()

## Haz de agente!

Antes de pasar a ver cómo se soluciona el entorno con Q-Learning, intenta hacer una política de manera manual como si fueras un agente.

Para ello, modifica el código de la función `politica_a_mano` anterior. Juega con el valor de las variables del estado para ir decidiendo si es mejor moverse a la izquieda (0) o a la derecha (1).

Programa la política según tus reglas, mira el retorno que obtienes y haz modificaciones para intentar optimizarlo.